In [3]:
from pypdf import PdfReader


In [4]:
pdf_path = pdf_path = "C:/Users/HP/Desktop/project1/rag-knowledge-assistant/data/uploaded_docs/Medical_book.pdf"

reader = PdfReader(pdf_path)

len(reader.pages)


637

In [5]:
print(len(reader.pages))

637


In [6]:
text = ""

for page in reader.pages:
    page_text = page.extract_text()
    if page_text:
        text += page_text + "\n"

len(text)


2630696

In [7]:
print(text[:1500])


The GALE
ENCYCLOPEDIA
of MEDICINE
SECOND EDITION
The GALE
ENCYCLOPEDIA
of MEDICINE
SECOND EDITION
JACQUELINE L. LONGE, EDITOR
DEIRDRE S. BLANCHFIELD, ASSOCIATE EDITOR
VOLUME
A-B
1
STAFF
Jacqueline L. Longe, Project Editor
Deirdre S. Blanchfield, Associate Editor
Christine B. Jeryan, Managing Editor
Donna Olendorf, Senior Editor
Stacey Blachford, Associate Editor
Kate Kretschmann, Melissa C. McDade, Ryan
Thomason, Assistant Editors
Mark Springer, Technical Specialist
Andrea Lopeman, Programmer/Analyst
Barbara J. Yarrow,Manager, Imaging and Multimedia
Content
Robyn V . Young,Project Manager, Imaging and
Multimedia Content
Dean Dauphinais, Senior Editor, Imaging and
Multimedia Content
Kelly A. Quin, Editor, Imaging and Multimedia Content
Leitha Etheridge-Sims, Mary K. Grimes, Dave Oblender,
Image Catalogers
Pamela A. Reed, Imaging Coordinator
Randy Bassett, Imaging Supervisor
Robert Duncan, Senior Imaging Specialist
Dan Newell, Imaging Specialist
Christine O’Bryan, Graphic Specialist
Mari

In [8]:
clean_text = text.replace("\n\n", "\n").strip()


In [9]:
def chunk_text(text: str, chunk_size: int = 500, overlap: int = 100):
    chunks = []
    start = 0
    text_length = len(text)

    while start < text_length:
        end = start + chunk_size
        chunk = text[start:end]
        chunks.append(chunk)
        start = end - overlap

    return chunks


In [10]:
chunks = chunk_text(clean_text)
len(chunks)


6577

In [11]:
print(chunks[0])
print(chunks[1])


The GALE
ENCYCLOPEDIA
of MEDICINE
SECOND EDITION
The GALE
ENCYCLOPEDIA
of MEDICINE
SECOND EDITION
JACQUELINE L. LONGE, EDITOR
DEIRDRE S. BLANCHFIELD, ASSOCIATE EDITOR
VOLUME
A-B
1
STAFF
Jacqueline L. Longe, Project Editor
Deirdre S. Blanchfield, Associate Editor
Christine B. Jeryan, Managing Editor
Donna Olendorf, Senior Editor
Stacey Blachford, Associate Editor
Kate Kretschmann, Melissa C. McDade, Ryan
Thomason, Assistant Editors
Mark Springer, Technical Specialist
Andrea Lopeman, Programmer/An
, Ryan
Thomason, Assistant Editors
Mark Springer, Technical Specialist
Andrea Lopeman, Programmer/Analyst
Barbara J. Yarrow,Manager, Imaging and Multimedia
Content
Robyn V . Young,Project Manager, Imaging and
Multimedia Content
Dean Dauphinais, Senior Editor, Imaging and
Multimedia Content
Kelly A. Quin, Editor, Imaging and Multimedia Content
Leitha Etheridge-Sims, Mary K. Grimes, Dave Oblender,
Image Catalogers
Pamela A. Reed, Imaging Coordinator
Randy Bassett, Imaging Supervisor
Robert Duncan

In [12]:
print(chunks[0][-120:])
print(chunks[1][:120])


n, Melissa C. McDade, Ryan
Thomason, Assistant Editors
Mark Springer, Technical Specialist
Andrea Lopeman, Programmer/An
, Ryan
Thomason, Assistant Editors
Mark Springer, Technical Specialist
Andrea Lopeman, Programmer/Analyst
Barbara J. Yar


In [13]:
import ollama

def embed_texts(texts):
    embeddings = []
    for text in texts:
        response = ollama.embeddings(
            model="nomic-embed-text",
            prompt=text
        )
        embeddings.append(response["embedding"])
    return embeddings


In [14]:
embeddings = embed_texts(chunks)

len(embeddings), len(embeddings[0])


(6577, 768)

In [15]:
type(embeddings[0])


list

In [16]:
list


list

In [17]:
import math

def cosine_similarity(vec1, vec2):
    dot_product = sum(a * b for a, b in zip(vec1, vec2))
    norm1 = math.sqrt(sum(a * a for a in vec1))
    norm2 = math.sqrt(sum(b * b for b in vec2))
    return dot_product / (norm1 * norm2)


In [18]:
query = "What are the symptoms of diabetes?"

query_embedding = embed_texts([query])[0]


In [19]:
scores = []

for i, chunk_embedding in enumerate(embeddings):
    score = cosine_similarity(query_embedding, chunk_embedding)
    scores.append((score, i))


In [20]:
scores.sort(reverse=True)


In [21]:
top_k = 3

for score, idx in scores[:top_k]:
    print(f"\nScore: {score:.4f}")
    print(chunks[idx][:500])



Score: 0.7100
occur.
DIABETES. Antiarrhythmic drugs may cause low
blood sugar, which can be a particular problem for peo-
ple with diabetes. Anyone with diabetes should be famil-
iar with the signs of low blood sugar (listed above) and
should check with his or her physician about what to do
if such symptoms occur.
PREGNANCY. The effects of taking antiarrhythmic
drugs in pregnancy have not been studied in humans. In
studies of laboratory animals, this medicine increased the
risk of miscarriage. In addition, so

Score: 0.7009
g any surgical or dental procedures or
receiving emergency treatment.
Antiarrhythmic drugs may cause low blood sugar in
some people. Anyone who experiences symptoms of low
blood sugar should eat or drink a food that contains sugar
and call a physician immediately. Signs of low blood
sugar are:
• anxiety
• confusion
• nervousness
• shakiness
• unsteady walk
• extreme hunger
• headache
• nausea
• drowsiness
• unusual tiredness or weakness
• fast heartbeat
• pale, coo

In [22]:
top_k = 3
top_chunks = [chunks[idx] for _, idx in scores[:top_k]]


In [23]:
context = "\n\n".join(top_chunks)

prompt = f"""
You are a helpful assistant.
Answer the question using ONLY the context below.
If the answer is not in the context, say "I don't know".

Context:
{context}

Question:
{query}

Answer:
"""


In [24]:
response = ollama.generate(
    model="llama3.2:3b",
    prompt=prompt
)

print(response["response"])


The symptoms listed in the context are not for diabetes itself, but rather for low blood sugar (hypoglycemia). However, according to the context, the symptoms of diabetes are:

• Blood sugar that remains high
• Hyperglycemia
• Damage or failure of the eyes
• Kidneys
• Nerves
• Heart
• Blood vessels

Note: These are not the specific symptoms listed for low blood sugar in the context.


In [28]:
import sys
import os

sys.path.append(os.path.abspath(".."))


In [ ]:
from app.core.rag import build_rag_index, retrieve

pdf_path = "C:/Users/HP/Desktop/project1/rag-knowledge-assistant/data/uploaded_docs/Medical_book.pdf"

chunks, embeddings = build_rag_index(pdf_path)

query = "What are the symptoms of diabetes?"
top_chunks = retrieve(query, chunks, embeddings)

for chunk in top_chunks:
    print(chunk[:400])


KeyboardInterrupt: 

: 